# Neural net

In [1]:
import pandas as pd
import numpy as np
import bq_helper
from bq_helper import BigQueryHelper
# https://www.kaggle.com/sohier/introduction-to-the-bq-helper-package
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Layer, Dense, Dropout, Bidirectional, Embedding, LSTM, GRU, BatchNormalization, TextVectorization, Flatten
from tensorflow.keras.optimizers import Adam
genomes = bq_helper.BigQueryHelper(active_project="bigquery-public-data",
                                   dataset_name="genomics_cannabis")

In [2]:
bq_assistant = BigQueryHelper("bigquery-public-data", "genomics_cannabis")
bq_assistant.list_tables()

['MNPR01_201703',
 'MNPR01_reference_201703',
 'MNPR01_transcriptome_201703',
 'cs10_gff',
 'cs3k_project_info',
 'cs3k_vcf_cs10_dv090',
 'sample_info_201703']

In [3]:
query = """SELECT
  variant_id, alternate_bases, quality, type, 
  AB, ABP, AC, AF, AN, AO, DP, DPB, DPRA, EPP, 
  EPPR, GTI, MQM, MQMR, NS, NUMALT, ODDS, PAO,
  PQA, PQR, PRO, QA, QR, RO, RPL, RPP, RPPR, 
  RPR, RUN, SAF, SAP, SAR, SRF, SRP, SRR
FROM
  `bigquery-public-data.genomics_cannabis.MNPR01_201703` v
limit 100000;"""
response = genomes.query_to_pandas_safe(query, max_gb_scanned=100)
response.head(10)

C:\Users\sharm\AppData\Roaming\Python\Python311\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,variant_id,alternate_bases,quality,type,AB,ABP,AC,AF,AN,AO,...,RPP,RPPR,RPR,RUN,SAF,SAP,SAR,SRF,SRP,SRR
0,CKXG8eKP9qOf8wESIGdpfDEwOTg0OTQ0OTl8Z2J8TU5QUj...,[ATTTTTTTA],14.63050,[del],[0.0],[0.0],[2],[1.0],2,[1],...,[5.18177],0.0,[0],[1],[1],[5.18177],[0],0,0.0,0
1,CKXG8eKP9qOf8wESIGdpfDEwOTg0OTU3Mjl8Z2J8TU5QUj...,[TGAAAAAAAT],22.95050,[del],[0.0],[0.0],[2],[1.0],2,[1],...,[5.18177],0.0,[0],[1],[1],[5.18177],[0],0,0.0,0
2,CKXG8eKP9qOf8wESIGdpfDEwOTg0OTcwNjJ8Z2J8TU5QUj...,[ACTA],19.64700,[snp],[0.0],[0.0],[2],[1.0],2,[1],...,[5.18177],0.0,[1],[1],[1],[5.18177],[0],0,0.0,0
3,CKXG8eKP9qOf8wESIGdpfDEwOTg0OTU1MDZ8Z2J8TU5QUj...,[ACACTCT],4.75779,[snp],[0.0],[0.0],[2],[1.0],2,[1],...,[5.18177],0.0,[0],[1],[0],[5.18177],[1],0,0.0,0
4,CKXG8eKP9qOf8wESIGdpfDEwOTg0OTUyNjd8Z2J8TU5QUj...,[ATTTTTAA],20.48920,[snp],[0.0],[0.0],[2],[1.0],2,[1],...,[5.18177],0.0,[0],[1],[1],[5.18177],[0],0,0.0,0
5,CKXG8eKP9qOf8wESIGdpfDEwOTg0OTY4MDl8Z2J8TU5QUj...,[GCTC],16.45330,[snp],[0.0],[0.0],[2],[1.0],2,[1],...,[5.18177],0.0,[0],[1],[0],[5.18177],[1],0,0.0,0
6,CKXG8eKP9qOf8wESIGdpfDEwOTg0ODYyNjd8Z2J8TU5QUj...,[ACTGAATGAATA],9.10780,[complex],[0.0],[0.0],[2],[1.0],2,[1],...,[5.18177],0.0,[0],[1],[0],[5.18177],[1],0,0.0,0
7,CKXG8eKP9qOf8wESIGdpfDEwOTg0OTU2NDR8Z2J8TU5QUj...,[GA],8.21977,[snp],[0.0],[0.0],[2],[1.0],2,[1],...,[5.18177],0.0,[0],[1],[0],[5.18177],[1],0,0.0,0
8,CKXG8eKP9qOf8wESIGdpfDEwOTg0OTY3NzF8Z2J8TU5QUj...,[TTTC],14.13530,[mnp],[0.0],[0.0],[2],[1.0],2,[1],...,[5.18177],0.0,[1],[1],[1],[5.18177],[0],0,0.0,0
9,CKXG8eKP9qOf8wESIGdpfDEwOTg0OTM5MDB8Z2J8TU5QUj...,[GT],21.22020,[snp],[0.0],[0.0],[2],[1.0],2,[1],...,[5.18177],0.0,[0],[1],[1],[5.18177],[0],0,0.0,0


In [4]:
response["DNA_A"] = response["alternate_bases"].str[0].str.count('A')
response["DNA_C"] = response["alternate_bases"].str[0].str.count('C')
response["DNA_G"] = response["alternate_bases"].str[0].str.count('G')
response["DNA_T"] = response["alternate_bases"].str[0].str.count('T')

In [5]:
proteins = ["AB", "ABP", "AC", "AF", "AO", "DPRA", "EPP", "MQM", "PAO",
            "PQA", "QA", "RPL", "RPP", "type",
            "RPR", "RUN", "SAF", "SAP", "SAR", 
]

for protein in proteins:
    response[protein] = response[protein].str[0]

In [6]:
dummies = pd.get_dummies(response["type"], dtype="int")

In [7]:
X = pd.concat([
    response[["AB", "ABP", "AC", "AF", "AN", "AO", "DP", "DPB", "DPRA", "EPP", 
              "EPPR", "GTI", "MQM", "MQMR", "NS", "NUMALT", "ODDS", "PAO",
              "PQA", "PQR", "PRO", "QA", "QR", "RO", "RPL", "RPP", "RPPR", 
              "RPR", "RUN", "SAF", "SAP", "SAR", "SRF", "SRP", "SRR"
    ]], 
    dummies
], axis=1)
y = response[["DNA_A", "DNA_C", "DNA_G", "DNA_T"]]

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

import torch
import torch.nn as nn

from tqdm.notebook import tnrange, tqdm_notebook

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.33, random_state=1337)

In [10]:
X.shape

(100000, 40)

In [11]:
X_train

,AB,ABP,AC,AF,AN,AO,DP,DPB,DPRA,EPP,...,SAP,SAR,SRF,SRP,SRR,complex,del,ins,mnp,snp
25842,0.0,0.0,2,1.0,2,2,2,2.00000,0,7.35324,...,7.35324,2,0,0.0,0,0,0,0,0,1
60834,0.0,0.0,2,1.0,2,2,2,2.00000,0,3.01030,...,3.01030,1,0,0.0,0,0,0,0,0,1
60689,0.0,0.0,2,1.0,2,2,2,2.00000,0,3.01030,...,3.01030,1,0,0.0,0,0,0,0,0,1
68241,0.0,0.0,2,1.0,2,2,2,2.00000,0,3.01030,...,3.01030,1,0,0.0,0,0,0,0,0,1
90328,0.0,0.0,2,1.0,2,3,3,3.00000,0,9.52472,...,3.73412,2,0,0.0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85159,0.0,0.0,2,1.0,2,2,2,2.00000,0,3.01030,...,3.01030,1,0,0.0,0,0,0,0,0,1
9448,0.0,0.0,2,1.0,2,2,3,3.00000,0,3.01030,...,3.01030,1,0,0.0,0,1,0,0,0,0
99164,0.0,0.0,2,1.0,2,2,2,2.00000,0,7.35324,...,7.35324,0,0,0.0,0,0,0,0,0,1
57533,0.0,0.0,2,1.0,2,2,2,1.33333,0,7.35324,...,7.35324,0,0,0.0,0,0,1,0,0,0


In [12]:
X_train = torch.tensor(X_train.to_numpy(dtype='float32'), dtype=torch.float32)
X_test = torch.tensor(X_test.to_numpy(dtype='float32'), dtype=torch.float32)

y_train = torch.tensor(y_train.to_numpy(dtype='float32'), dtype=torch.long)
y_test = torch.tensor(y_test.to_numpy(dtype='float32'), dtype=torch.long)

In [13]:
x = X_train[0:8, :]
target = y_train[0:8].to(torch.float32)

neural_net = nn.Sequential(nn.Linear(40, 4),
                           nn.ReLU(),
                           nn.Linear(4, 1))
mse_loss = nn.MSELoss()

optimizer = torch.optim.SGD(neural_net.parameters(), lr=0.1)

output_values = neural_net(x)
loss = mse_loss(output_values, target)

for _ in range(0, 1000):
    loss.backward()
    optimizer.step()

    new_output = neural_net(x)
    new_loss = mse_loss(new_output, target)
    print(f"Old loss: {loss}; New loss: {new_loss}")
    loss = new_loss

c:\ProgramData\anaconda3\envs\ml_env\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([8, 4])) that is different to the input size (torch.Size([8, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Old loss: 9.468994140625; New loss: 0.4142494201660156
Old loss: 0.4142494201660156; New loss: 0.7283279299736023
Old loss: 0.7283279299736023; New loss: 1.5045788288116455
Old loss: 1.5045788288116455; New loss: 2.1530513763427734
Old loss: 2.1530513763427734; New loss: 2.1809065341949463
Old loss: 2.1809065341949463; New loss: 1.56697416305542
Old loss: 1.56697416305542; New loss: 0.777843177318573
Old loss: 0.777843177318573; New loss: 0.41325289011001587
Old loss: 0.41325289011001587; New loss: 0.7502920627593994
Old loss: 0.7502920627593994; New loss: 1.532810926437378
Old loss: 1.532810926437378; New loss: 2.166095018386841
Old loss: 2.166095018386841; New loss: 2.1688485145568848
Old loss: 2.1688485145568848; New loss: 1.5389785766601562
Old loss: 1.5389785766601562; New loss: 0.7551866769790649
Old loss: 0.7551866769790649; New loss: 0.41315436363220215
Old loss: 0.41315436363220215; New loss: 0.7728263139724731
Old loss: 0.7728263139724731; New loss: 1.5608515739440918
Old los

This is a low amount of loss. 